__author__ = 'Ricardo Pasquini'

# Counting tweets at the hexagon level. Complete procedure

Overview: This code counts georeferenced tweets at the hexagon level. Hexagons are uniquely identified in space. We use Uber's H3 library for this purpose.  
The code  also performs a home location prediction for each user. This allows later counting tweets by whether they were tweeted by residents, residents of neighbors hexagons, or non-residents.
The project was designed to be scalable to millions of tweets. The process is done with MongoDB.

By default the project generates the following collections on the Mongo database:

tweets: Collection containing tweets. This collection is modified by incorporating the corresponding hex id. \
users: Collection containing users. This collection is modified by incorporating the home location data.\
hexcounts: Collection containing counts of tweets.


In [1]:
import sys
sys.path.append("../") 
import databasepopulation
import communicationwmongo as commu
import home_location as home
import analysis as a
import pymongo
import pandas as pd
import my_h3_functions as myh3

In [2]:
import importlib
importlib.reload(databasepopulation)
importlib.reload(home)
importlib.reload(a)
importlib.reload(myh3)

<module 'my_h3_functions' from '..\\my_h3_functions.py'>

# Counting tweets at the hexagon level. Complete procedure

This version of the code assumes that a different database will be used for each city.


If necessary check your current location with the following command:

In [3]:
pwd

'C:\\Users\\emman\\Documents\\git\\twitter_and_displacement\\notebooks'

# 0. Connect to Mongo and define a specific database

By default connect to Mongo running on a localhost. 
Requires manually changing the parameters if working on a cloud environment

I will use the name buenosaires as the db example. Recall each database for each city.


In [4]:
db=commu.connecttoLocaldb(database='twitter_bog')

In [5]:
db.Twitter_Data

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'twitter_bog'), 'Twitter_Data')

# 1. Raw data to Mongo
This function populates the files containing tweets into Mongo.
Assumes files are named like: ba_2012.csv, etc.
Choose start and end years.


In [ ]:
databasepopulation.populatetweets(db, path='/Users/emman/Box/Twitter data/Latin America/Bogota/', cityprefix='bo', yearstart=2012, yearend=2012)

In [ ]:
databasepopulation.populatetweets(db, path='/Users/emman/Box/Twitter data/Latin America/Bogota/', cityprefix='bo', yearstart=2015, yearend=2015)

In [ ]:
db.tweets.update({},{'$rename':{"latitude":"lat","longitude":"lon"}}, False,True,True)

# 2. Adding Hexids to tweets 
This function includes hexagons ids to each tweet.  

In [6]:
databasepopulation.addhexjob(db)

 iter: 1  time: 0.4697873592376709
 iter: 2  time: 0.40143537521362305
 iter: 3  time: 0.2792544364929199
 iter: 4  time: 0.23536944389343262
 iter: 5  time: 0.18849754333496094
 iter: 6  time: 0.17817115783691406
 iter: 7  time: 0.2293870449066162
 iter: 8  time: 0.27100467681884766
 iter: 9  time: 0.3171510696411133
 iter: 10  time: 0.3171520233154297
 iter: 11  time: 0.36801815032958984
 iter: 12  time: 0.19148731231689453
 iter: 13  time: 0.1894056797027588
 iter: 14  time: 0.22539830207824707
 iter: 15  time: 0.2593064308166504
 iter: 16  time: 0.2822449207305908
 iter: 17  time: 0.22403979301452637
 iter: 18  time: 0.19946622848510742
 iter: 19  time: 0.19466423988342285
 iter: 20  time: 0.2143995761871338
 iter: 21  time: 0.3690149784088135
 iter: 22  time: 0.28124427795410156
 iter: 23  time: 0.24534392356872559
 iter: 24  time: 0.2872316837310791
 iter: 25  time: 0.2794802188873291
 iter: 26  time: 0.19682788848876953
 iter: 27  time: 0.20345544815063477
 iter: 28  time: 0.167

 iter: 223  time: 0.1625652313232422
 iter: 224  time: 0.17253899574279785
 iter: 225  time: 0.19148850440979004
 iter: 226  time: 0.17417478561401367
 iter: 227  time: 0.16057038307189941
 iter: 228  time: 0.1629786491394043
 iter: 229  time: 0.16251540184020996
 iter: 230  time: 0.16638684272766113
 iter: 231  time: 0.17453312873840332
 iter: 232  time: 0.1934833526611328
 iter: 233  time: 0.17375397682189941
 iter: 234  time: 0.17304563522338867
 iter: 235  time: 0.16848182678222656
 iter: 236  time: 0.16655445098876953
 iter: 237  time: 0.1685495376586914
 iter: 238  time: 0.17553019523620605
 iter: 239  time: 0.21058869361877441
 iter: 240  time: 0.16356539726257324
 iter: 241  time: 0.16057085990905762
 iter: 242  time: 0.16564249992370605
 iter: 243  time: 0.16451668739318848
 iter: 244  time: 0.17955636978149414
 iter: 245  time: 0.19146132469177246
 iter: 246  time: 0.17600798606872559
 iter: 247  time: 0.15513372421264648
 iter: 248  time: 0.17465972900390625
 iter: 249  time

 iter: 442  time: 0.17003488540649414
 iter: 443  time: 0.21796631813049316
 iter: 444  time: 0.18052411079406738
 iter: 445  time: 0.16599512100219727
 iter: 446  time: 0.16176915168762207
 iter: 447  time: 0.16409564018249512
 iter: 448  time: 0.15719246864318848
 iter: 449  time: 0.16830873489379883
 iter: 450  time: 0.1988532543182373
 iter: 451  time: 0.16360807418823242
 iter: 452  time: 0.16077566146850586
 iter: 453  time: 0.16661572456359863
 iter: 454  time: 0.1924905776977539
 iter: 455  time: 0.16111207008361816
 iter: 456  time: 0.17760324478149414
 iter: 457  time: 0.15610623359680176
 iter: 458  time: 0.15386033058166504
 iter: 459  time: 0.16626286506652832
 iter: 460  time: 0.154524564743042
 iter: 461  time: 0.1643514633178711
 iter: 462  time: 0.1636962890625
 iter: 463  time: 0.18351507186889648
 iter: 464  time: 0.16443681716918945
 iter: 465  time: 0.1525886058807373
 iter: 466  time: 0.15865349769592285
 iter: 467  time: 0.16678857803344727
 iter: 468  time: 0.15

 iter: 661  time: 0.19348526000976562
 iter: 662  time: 0.2040238380432129
 iter: 663  time: 0.23835325241088867
 iter: 664  time: 0.17354249954223633
 iter: 665  time: 0.17411255836486816
 iter: 666  time: 0.17752599716186523
 iter: 667  time: 0.16957950592041016
 iter: 668  time: 0.18750476837158203
 iter: 669  time: 0.17054319381713867
 iter: 670  time: 0.15298008918762207
 iter: 671  time: 0.18951749801635742
 iter: 672  time: 0.15860342979431152
 iter: 673  time: 0.17694973945617676
 iter: 674  time: 0.1829395294189453
 iter: 675  time: 0.2538292407989502
 iter: 676  time: 0.1999197006225586
 iter: 677  time: 0.17054343223571777
 iter: 678  time: 0.15658116340637207
 iter: 679  time: 0.15458941459655762
 iter: 680  time: 0.15262556076049805
 iter: 681  time: 0.18605303764343262
 iter: 682  time: 0.15189695358276367
 iter: 683  time: 0.14860057830810547
 iter: 684  time: 0.14961552619934082
 iter: 685  time: 0.16057324409484863
 iter: 686  time: 0.16057038307189941
 iter: 687  time

 iter: 879  time: 0.22144412994384766
 iter: 880  time: 0.18639421463012695
 iter: 881  time: 0.18105578422546387
 iter: 882  time: 0.16228437423706055
 iter: 883  time: 0.16318726539611816
 iter: 884  time: 0.17994403839111328
 iter: 885  time: 0.20323872566223145
 iter: 886  time: 0.16614508628845215
 iter: 887  time: 0.1556391716003418
 iter: 888  time: 0.15603256225585938
 iter: 889  time: 0.1586165428161621
 iter: 890  time: 0.1545858383178711
 iter: 891  time: 0.15757012367248535
 iter: 892  time: 0.17706894874572754
 iter: 893  time: 0.14966988563537598
 iter: 894  time: 0.15685415267944336
 iter: 895  time: 0.15658354759216309
 iter: 896  time: 0.15459108352661133
 iter: 897  time: 0.16178417205810547
 iter: 898  time: 0.15731191635131836
 iter: 899  time: 0.17841124534606934
 iter: 900  time: 0.1800692081451416
 iter: 901  time: 0.16057538986206055
 iter: 902  time: 0.16455435752868652
 iter: 903  time: 0.1555941104888916
 iter: 904  time: 0.15148401260375977
 iter: 905  time:

...

 iter: 2800  time: 0.251392126083374
 iter: 2801  time: 0.2575037479400635
 iter: 2802  time: 0.23288655281066895
 iter: 2803  time: 0.26453733444213867
 iter: 2804  time: 0.25033068656921387
 iter: 2805  time: 0.26927947998046875
 iter: 2806  time: 0.24140334129333496
 iter: 2807  time: 0.24467873573303223
 iter: 2808  time: 0.2509121894836426
 iter: 2809  time: 0.23498320579528809
 iter: 2810  time: 0.2518937587738037
 iter: 2811  time: 0.2672915458679199
 iter: 2812  time: 0.22636866569519043
 iter: 2813  time: 0.235365629196167
 iter: 2814  time: 0.23237895965576172
 iter: 2815  time: 0.23886919021606445
 iter: 2816  time: 0.2413499355316162
 iter: 2817  time: 0.23591113090515137
 iter: 2818  time: 0.24642658233642578
 iter: 2819  time: 0.24022483825683594
 iter: 2820  time: 0.2602956295013428
 iter: 2821  time: 0.2713029384613037
 iter: 2822  time: 0.32114076614379883
 iter: 2823  time: 0.2713623046875
 iter: 2824  time: 0.26895689964294434
 iter: 2825  time: 0.3193173408508301
 i

 iter: 3013  time: 0.22739243507385254
 iter: 3014  time: 0.26206016540527344
 iter: 3015  time: 0.23636913299560547
 iter: 3016  time: 0.2520270347595215
 iter: 3017  time: 0.23242974281311035
 iter: 3018  time: 0.24482274055480957
 iter: 3019  time: 0.23839306831359863
 iter: 3020  time: 0.21938443183898926
 iter: 3021  time: 0.23746871948242188
 iter: 3022  time: 0.26833176612854004
 iter: 3023  time: 0.2224445343017578
 iter: 3024  time: 0.22871160507202148
 iter: 3025  time: 0.22237515449523926
 iter: 3026  time: 0.21941328048706055
 iter: 3027  time: 0.2104356288909912
 iter: 3028  time: 0.2488727569580078
 iter: 3029  time: 0.29681849479675293
 iter: 3030  time: 0.25953006744384766
 iter: 3031  time: 0.2591276168823242
 iter: 3032  time: 0.2388896942138672
 iter: 3033  time: 0.29950594902038574
 iter: 3034  time: 0.2732675075531006
 iter: 3035  time: 0.32018136978149414
 iter: 3036  time: 0.22942686080932617
 iter: 3037  time: 0.2121257781982422
 iter: 3038  time: 0.232184886932

 iter: 3226  time: 0.3106985092163086
 iter: 3227  time: 0.28977394104003906
 iter: 3228  time: 0.2786545753479004
 iter: 3229  time: 0.2922170162200928
 iter: 3230  time: 0.30521535873413086
 iter: 3231  time: 0.29967784881591797
 iter: 3232  time: 0.3007051944732666
 iter: 3233  time: 0.30720996856689453
 iter: 3234  time: 0.29334235191345215
 iter: 3235  time: 0.2832760810852051
 iter: 3236  time: 0.27426862716674805
 iter: 3237  time: 0.28971171379089355
 iter: 3238  time: 0.3131997585296631
 iter: 3239  time: 0.30608510971069336
 iter: 3240  time: 0.3201460838317871
 iter: 3241  time: 0.31914401054382324
 iter: 3242  time: 0.3206522464752197
 iter: 3243  time: 0.4532613754272461
 iter: 3244  time: 0.3196876049041748
 iter: 3245  time: 0.28112244606018066
 iter: 3246  time: 0.320265531539917
 iter: 3247  time: 0.29720616340637207
 iter: 3248  time: 0.29677844047546387
 iter: 3249  time: 0.3029356002807617
 iter: 3250  time: 0.2942194938659668
 iter: 3251  time: 0.29328036308288574


 iter: 3439  time: 0.23932528495788574
 iter: 3440  time: 0.2513294219970703
 iter: 3441  time: 0.24138879776000977
 iter: 3442  time: 0.2503364086151123
 iter: 3443  time: 0.2765669822692871
 iter: 3444  time: 0.3725473880767822
 iter: 3445  time: 0.2562844753265381
 iter: 3446  time: 0.23879122734069824
 iter: 3447  time: 0.25528454780578613
 iter: 3448  time: 0.24065923690795898
 iter: 3449  time: 0.2361750602722168
 iter: 3450  time: 0.2762298583984375
 iter: 3451  time: 0.2403552532196045
 iter: 3452  time: 0.25638270378112793
 iter: 3453  time: 0.2483837604522705
 iter: 3454  time: 0.23603129386901855
 iter: 3455  time: 0.2433161735534668
 iter: 3456  time: 0.2373964786529541
 iter: 3457  time: 0.2666652202606201
 iter: 3458  time: 0.22542834281921387
 iter: 3459  time: 0.22539234161376953
 iter: 3460  time: 0.2413642406463623
 iter: 3461  time: 0.22679352760314941
 iter: 3462  time: 0.24135637283325195
 iter: 3463  time: 0.23340415954589844
 iter: 3464  time: 0.2676990032196045


AutoReconnect: localhost:27017: [WinError 10054] An existing connection was forcibly closed by the remote host

In [ ]:
# the following is a check and a fix because I think the previous process did not run completely, probably a bug.
# are there any other documents that were not processed?
len(list(db.tweets.find({ "hex" : { "$exists" : False } })))

In [ ]:
#Process any missing document
cursor=db.tweets.find({ "hex" : { "$exists" : False } })
df = pd.DataFrame(list(cursor))
requests = databasepopulation.add_hexs_and_prepare_bulk_request(df, dataformat='raw')
try:
    db.tweets.bulk_write(requests, ordered=False)
except BulkWriteError as bwe:
    print(bwe.details)

# 3. Database performance task: Add indexes to tweets

In [ ]:
databasepopulation.create_indexes(db)

# 4. Populate Users Collection

In [ ]:
databasepopulation.populate_users_collection(db)

# 5. Find Home Job for each user id in the database

In [ ]:
print('Users with home location identified', db.users.count_documents({'hex9': { '$exists': True} }))


# 6 Generate a Hex-level collection including all hexs in the database

By default the collection include as ids only the ids of hexagons at resolution 9
Note that many types of hexogons could coexist in a same collection because hex identifiers are unique. This is not implemented in the current version of the code, though.

In [ ]:
databasepopulation.populate_hexcounts_collection(db)

# 7 Count tweets in each hex by residents and non-residents

In [ ]:
import analysis as a

In [ ]:
a.countandpopulatejob(db)

# 8 Query to the DB to create the counts dataframe 

In [ ]:
import time 
start=time.time()
df=a.hexcountsresults_to_df(db, save=False)
print(time.time()-start)
# if save=True then saves resulting dataframe a pickle in ./hexcountsdf.pkl

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#df.to_csv('C:/Users/Emman/Desktop/Bogota_Hexes_RP.csv')

# Funcions for Spatial Analysis
### A. Transform the dataframe with hexids into a geodataframe with hexagons as geometries

In [ ]:
hexgdf = myh3.df_with_hexid_to_gdf(df, hexcolname='_id')
hexgdf.plot()

In [ ]:
hexgdf.head()

In [ ]:
hexgdf = myh3.df_with_hexid_to_gdf(df, hexcolname='_id')
hexgdf.plot()

## B. Transform the dataframe with hexids into a geodataframe with centoids points as geometries

#### Points can be used for spatial joins.

In [ ]:
centroidsgdf = myh3.df_with_hexid_to_centroids_gdf(hexgdf, hexcolname='_id')
centroidsgdf.rotate(270, origin = (0,0), use_radians=False).plot()

In [ ]:
smooth_hexgdf = myh3.kring_smoother(hexgdf, hexcolname='_id',  metric_col='nonresidents')

In [ ]:
smooth_hexgdf.info()

In [ ]:
smooth_hexgdf.plot(column='nonresidents')